In [2]:
import pandas as pd

In [4]:
from py2neo import Graph, Node, Relationship

In [5]:
from py2neo.bulk import merge_nodes, create_nodes, create_relationships, merge_relationships

In [6]:
from math import ceil
from time import time
import gc
import subprocess

## Connect to graph

In [7]:
uri = "bolt://localhost:7687"
user = "neo4j"
pword = "localadmin"

In [8]:
graph = Graph(uri, user=user, password=pword)

## Some utilities

In [9]:
count_nodes = lambda node_type: graph.run(f"match (n:{node_type}) return count(n) as count").evaluate()

In [10]:
count_rels = lambda relationship: graph.run(f"match ()-[r:{relationship}]->() return count(r) as count").evaluate()

In [129]:
# current counts
number_judges = count_nodes("Judge")
number_cases = count_nodes("Case")
number_acts = count_nodes("Act")

In [130]:
number_judge_rel = count_rels("JUDGED")
number_act_rel = count_rels("USES_ACT")

In [131]:
print(f"In graph: judges: {number_judges}, cases: {number_judges}, acts: {number_acts}, judge-case rels: {number_judge_rel}, case-act rels: {number_act_rel}")

In graph: judges: 98478, cases: 98478, acts: 30847, judge-case rels: 2875323, case-act rels: 18377312


## Load up judges and add them to the graph

In [ ]:
jdf = pd.read_csv('../data/judges_clean.csv')

In [ ]:
jdf["start_date"] = pd.to_datetime(jdf["start_date"])
jdf["end_date"] = pd.to_datetime(jdf["end_date"])

In [ ]:
jdf.describe()

In [ ]:
jdf.head(n=10)

In [ ]:
def manual_commit_judges(start_row=0, end_row=len(jdf)):
    keys = ["judge_id", "judge_position", "judge_female", "judge_start", "judge_end"]
    jslice = jdf[start_row:end_row].iterrows()
    s1 = time()
    data = [
        [row["ddl_judge_id"], 
         row["judge_position"], 
         row["female_judge"],
         row["start_date"].strftime("%Y-%m-%d"),
         "" if pd.isnull(row["end_date"]) else row["end_date"].strftime("%Y-%m-%d")
        ]
    for index, row in jslice]
    e1 = time()
    #print("Time to process list: ", e1 - s1)
    s2 = time()
    create_nodes(graph.auto(), data, labels={"Judge"}, keys=keys)
    e2 = time()
    # print("Time to do TX: ", e2 - s2)

In [ ]:
def load_judges(batch_size=10000, delete_first=False, add_index=True):
    gc.collect() # just being cautious, given size of things being handed around
    if delete_first:
        graph.run("match (n: Judge) detach delete n")
    for batch in range(0, ceil(len(jdf)/batch_size)):
        print("Adding judges, batch: ", batch)
        manual_commit_judges(start_row=batch * batch_size, end_row = (batch + 1) * batch_size)
    print("Completed, number judges: ", graph.nodes.match("Judge").count())
    if add_index:
        graph.run("CREATE CONSTRAINT IF NOT EXISTS idx_judge_id ON (judge:Judge) ASSERT judge.judge_id IS UNIQUE")

In [ ]:
# not necessary, but would be analogue of above
load_judges_query = """
    load csv with headers from "file:///judges_clean.csv" as row
    with 
        toInteger(row.ddl_judge_id) as judge_id, 
        row.judge_position as judge_position, 
        toInteger(row.state_code) as judge_state_code,
        toInteger(row.dist_code) as judge_dist_code,
        toInteger(row.court_no) as judge_court_no,
        row.female_judge as judge_female,
        apoc.date.parse(row.start_date, "ms", "dd-MM-yyyy") as judgeStartMs,
        apoc.date.parse(row.end_date, "ms", "dd-MM-yyyy") as judgeEndMs,
    return judge_id, judget_position, judge_state_code, judge_dist_code, judge_cour_no, judge_female limit 10;
"""

## Now load some cases

In [61]:
def count_total_rows(csv_path):
    with open(csv_path) as csvfile:
        total_row_count = sum(1 for row in csvfile)
        return total_row_count

In [43]:
def load_cases(case_file, start_row=0, number_rows=1e6):
    cdf = pd.read_csv(case_file, nrows=number_rows, skiprows=range(1, int(start_row)))
    cdf["date_of_filing"] = pd.to_datetime(cdf["date_of_filing"])
    cdf["date_of_decision"] = pd.to_datetime(cdf["date_of_decision"], errors='coerce')
    return cdf

In [44]:
# slightly heavy, given size
total_cases = count_total_cases('../data/cases/cases_2018.csv')

In [45]:
cases_in_graph = count_nodes("Case")
print("Number of cases in graph: ", total_cases, " and in this graph: ", cases_in_graph)

Number of cases in graph:  13724300  and in this graph:  2999999


In [46]:
cdf = load_cases('../data/cases/cases_2018.csv', start_row=2e6, number_rows=1e6)

In [47]:
cdf.head()

,ddl_case_id,year,state_code,dist_code,court_no,judge_position,female_defendant,female_adv_def,female_adv_pet,type_name,purpose_name,disp_name,date_of_filing,date_of_decision,date_first_list,date_last_list,date_next_list
0,10-03-27-232301013862018,2018,10,3,27,judicial magistrate court,0 male,-9999,0,6950,450.0,17,2018-09-06,2018-09-08,2018-09-08,2018-09-08,2018-09-08
1,10-27-10-232300002762018,2018,10,27,10,judicial magistrate court,0 male,-9999,0,6950,277.0,17,2018-07-14,2018-07-14,2018-07-14,2018-07-14,2018-07-14
2,14-20-01-203400003012018,2018,14,20,1,district and sessions court,0 male,1,0,6680,5877.0,17,2018-11-26,2019-09-18,2018-11-26,2019-09-18,2019-09-18
3,16-10-02-204300000422018,2018,16,10,2,district and sessions court,1 female,-9999,-9999,7067,5053.0,17,2018-03-09,2018-12-10,2018-03-13,2018-12-10,2018-12-10
4,10-15-27-232320003462018,2018,10,15,27,judicial magistrate court,0 male,-9999,0,6950,450.0,17,2018-09-08,2018-09-08,2018-09-08,2018-09-08,2018-09-08


In [48]:
def manual_commit_cases(df, start_row=0, end_row=len(cdf), use_merge=True):
    keys = ["case_id", "year", "state_code", "dist_code", "court_no", "judge_position", "date_of_filing", "date_of_decision"]
    cslice = df[start_row:end_row].iterrows()
    s1 = time()
    data = [
        [row["ddl_case_id"], 
         row["year"], 
         row["state_code"],
         row["dist_code"],
         row["court_no"],
         row["judge_position"],
         row["date_of_filing"].strftime("%Y-%m-%d"),
         "" if pd.isnull(row["date_of_decision"]) else row["date_of_decision"].strftime("%Y-%m-%d")
        ]
    for index, row in cslice]
    e1 = time()
    #print("Time to process list: ", e1 - s1)
    s2 = time()
    if use_merge:
        merge_nodes(graph.auto(), data, merge_key=("Case", "case_id"), labels={"Case"}, keys=keys)
    else:
        create_nodes(graph.auto(), data, labels={"Case"}, keys=keys)
    e2 = time()
    print("Time to do TX: ", e2 - s2)

In [49]:
def commit_cases_df(cdf, batch_size=50000, delete_first=False, add_index=False):
    gc.collect()
    if delete_first:
        graph.run("match (n: Case) detach delete n")
    
    for batch in range(0, ceil(len(cdf)/batch_size)):
        print("Loading cases, batch: ", batch)
        manual_commit_cases(cdf, start_row=batch * batch_size, end_row = (batch + 1) * batch_size)
    
    if add_index: 
        graph.run("CREATE CONSTRAINT IF NOT EXISTS idx_case_id ON (case:Case) ASSERT case.case_id IS UNIQUE")

In [50]:
number_cases = count_nodes("Case")

In [ ]:
while number_cases < total_cases:
    start_row = number_cases + 1
    print("Loading cases from: ", start_row)
    cdf = load_cases('../data/cases/cases_2018.csv', start_row=start_row, number_rows=1e6)
    commit_cases_df(cdf, batch_size=50000)
    number_cases = count_nodes("Case")

# while number_in_graph < number_relationships and i < max_iter:
#     number_in_graph = add_batch_judge_relationships(minor_batch_size=200, major_batch_size=100000, use_merge=False)
#     i += 1

## Heavy lift: judge-case relationships

In [11]:
all_case_ids = pd.read_csv('../data/cases/cases_2018.csv', usecols=['ddl_case_id'])

In [12]:
df = pd.read_csv('../data/keys/judge_case_merge_key.csv')

In [13]:
len(df)

12901146

In [14]:
sdf = df[df.ddl_case_id.isin(all_case_ids.ddl_case_id)]

In [15]:
sdf.head()

,ddl_case_id,ddl_filing_judge_id,ddl_decision_judge_id
0,01-01-01-201900000022018,5.0,5.0
2,01-01-01-201900000032018,94.0,94.0
4,01-01-01-201900000042018,156.0,156.0
6,01-01-01-201900000052018,5.0,5.0
8,01-01-01-201900000062018,5.0,6.0


In [16]:
dedup_sdf = sdf[["ddl_case_id", "ddl_filing_judge_id"]].drop_duplicates()
print("Without dups: ", len(dedup_sdf), " vs raw: ", len(sdf))

Without dups:  2983723  vs raw:  2983723


In [17]:
number_relationships = len(sdf)
number_in_graph = count_rels("JUDGED")
print("Number relationships in data: ", number_relationships, " and in graph: ", number_in_graph)

Number relationships in data:  2983723  and in graph:  370636


In [56]:
# Note: use merge seems to return a lot of false positives on supposed duplicates, so defaulting to false for now
# ensure to do a thorough search to make sure no false duplicates later
def add_batch_judge_relationships(minor_batch_size=20, major_batch_size=10000, index_offset=0, use_merge=True):
    start_index = count_rels("JUDGED") + index_offset
    end_index = start_index + major_batch_size
    print(f"Adding relationships from {start_index} to {end_index}")
    data = [
        (row["ddl_filing_judge_id"], { "type": "FILING_JUDGE" }, row["ddl_case_id"])
        for index, row in sdf[start_index:end_index].iterrows()
    ]
    print("Numer of rows to add: ", len(data))
    print("Minor iterations: ", ceil(len(data) / minor_batch_size))

    for i in range(ceil(len(data) / minor_batch_size)):
        this_batch = data[i * minor_batch_size:(i + 1) * minor_batch_size]
        if i % 10 == 0:
            print(".", end="")

        if use_merge:
            merge_relationships(graph.auto(), this_batch,
                               "JUDGED", start_node_key=("Judge", "judge_id"), end_node_key=("Case", "case_id"))
        else:
            create_relationships(graph.auto(), this_batch, 
                                 "JUDGED", start_node_key=("Judge", "judge_id"), end_node_key=("Case", "case_id"))

    number_in_graph = count_rels("JUDGED")
    print("Completed a major addition, number in graph now: ", number_in_graph)
    return number_in_graph

In [57]:
number_in_graph = count_rels("JUDGED")
print("Number in graph: ", number_in_graph)

Number in graph:  811479


In [58]:
add_batch_judge_relationships(minor_batch_size=200, major_batch_size=100000, use_merge=True)

Adding relationships from 811479 to 911479
Numer of rows to add:  100000
Minor iterations:  500
..................................................Completed a major addition, number in graph now:  883603


883603

In [ ]:
i = 0
max_iter = 20
while number_in_graph < number_relationships and i < max_iter:
    number_in_graph = add_batch_judge_relationships(minor_batch_size=200, major_batch_size=100000, use_merge=False)
    i += 1

In [60]:
print("In graph: ", number_in_graph, " and in frame: ", len(sdf))

In graph:  2875323  and in frame:  2983723


## Load in the acts and sections

In [ ]:
def add_acts_to_graph(df, row_keys, node_keys, start_row=0, end_row=None, create_index=False):
    cslice = df[start_row:end_row].iterrows()
    data = [[row[key] for key in row_keys] for index, row in cslice]
    print("Assembled list, adding to graph")
    create_nodes(graph.auto(), data, labels={"Act"}, keys=node_keys)
    if create_index:
        graph.run("create constrain idx_act_id on (a:Act) assert a.act_id IS UNIQUE")

In [ ]:
acts = pd.read_csv('../data/keys/act_key.csv')
acts = acts[3:] # first rows are NA and ' and "
acts.head()

In [ ]:
act_df_keys = ["act", "count", "act_s"]
node_keys = ["act_id", "total_count", "act_sum"]

In [ ]:
add_acts_to_graph(df=acts, row_keys=act_df_keys, node_keys=node_keys)

In [ ]:
len(acts)

In [ ]:
# there are _a lot_ of these, and all are central, so distort things, so remove
criminal_procedure_variants = [
    "CODE OF CRIMINAL PROCEDURE, 1973",
    "Code of Criminal Procedure, 1973",
    "Code of Criminal Procedure 1973",
    "CODE OF CRIMINAL PROCEDURE",
    "Criminal Procedure Code",
    "Code of Criminal Procedure, 1973 1974",
    "CodeofCriminalProcedure",
    "Cr.P.C. ",
    "Code of Criminal Procedure",
    "2.Code of Criminal Procedure, 1973",
    "Cr.P.C.",
    "Cr.P.C",
    "Cr.P.c",
    "CR.P.C"
]

## Now do act-section relationships

In [62]:
total_act_section_rels = count_total_rows('../data/acts_sections.csv')

In [64]:
rels_in_graph = count_rels('USES_ACT')
print('Number of total relationships: ', total_act_section_rels, ' and in graph: ', rels_in_graph)

Number of total relationships:  76811358  and in graph:  84706


In [89]:
def load_act_section_rels(case_df, start_row=0, number_rows=1e6):
    act_case_df = pd.read_csv('../data/acts_sections.csv', nrows=number_rows, skiprows=range(1, int(start_row)))
    adf = act_case_df[act_case_df.ddl_case_id.isin(case_df.ddl_case_id)]
    return adf

In [120]:
def add_batch_act_relationships(adf, relationship_type="USES_ACT", df_start=0, minor_batch_size=20, major_batch_size=1000, use_merge=True):
    number_in_graph = count_rels(relationship_type)
    start_index = int(df_start)
    end_index = max(len(adf), int(start_index + major_batch_size))
    print(f"Adding relationships of type {relationship_type} from {start_index} to {end_index}")
    data = [
        (row["ddl_case_id"], { "type": relationship_type }, row["act"])
        for index, row in adf[start_index:end_index].iterrows()
    ]
    
    start_time = time()
    for i in range(ceil(len(data) / minor_batch_size)):
        if i % 10 == 0:
            print(".", end="")
            
        create_relationships(graph.auto(), 
                             data[i * minor_batch_size:(i + 1) * minor_batch_size], 
                             relationship_type, 
                             start_node_key=("Case", "case_id"), 
                             end_node_key=("Act", "act_id"))
    
    number_in_graph = count_rels(relationship_type)
    print("\nCompleted a major addition, number in graph now: ", number_in_graph, " took: ", int(time() - start_time))
    return number_in_graph, end_index

In [124]:
max_df_loads = 10
df_read_start = number_in_graph
rows_per_df = 5e6
i = 0

In [125]:
print('Would start from: ', number_in_graph)
last_stop = 60020612 # where last stopped - a little manual for now
df_read_start = int(last_stop + rows_per_df)
print('So in fact:', df_read_start)

Would start from:  16628617
So in fact: 65020612


In [126]:
while number_in_graph < total_act_section_rels and df_read_start < total_act_section_rels and i < max_df_loads:
    print('Initiating outer loop, loading dataframe, from row: ', df_read_start)
    adf = load_act_section_rels(all_case_ids, start_row=df_read_start, number_rows=rows_per_df)
    print('Length of this relationship frame: ', len(adf))
    end_index = 0
    while end_index < len(adf):
        number_in_graph, end_index = add_batch_act_relationships(
            adf, minor_batch_size=int(1e3), major_batch_size=int(1e6), df_start=end_index)
        print('Completed major batch load, end index: ', end_index)
    
    print('Completed loading this DF')
    number_in_graph = count_rels('USES_ACT')
    df_read_start += rows_per_df # to avoid the duplicates (given adf length < df read length)
    i += 1

Initiating outer loop, loading dataframe, from row:  65020612
Length of this relationship frame:  905307
Adding relationships of type USES_ACT from 0 to 1000000
...........................................................................................
Completed a major addition, number in graph now:  17532478  took:  55
Completed major batch load, end index:  1000000
Completed loading this DF
Initiating outer loop, loading dataframe, from row:  70020612.0
Length of this relationship frame:  613285
Adding relationships of type USES_ACT from 0 to 1000000
..............................................................
Completed a major addition, number in graph now:  18144663  took:  36
Completed major batch load, end index:  1000000
Completed loading this DF
Initiating outer loop, loading dataframe, from row:  75020612.0
Length of this relationship frame:  233205
Adding relationships of type USES_ACT from 0 to 1000000
........................
Completed a major addition, number in graph n

In [122]:
print(total_act_section_rels)

76811358
